# Clustering of weather data to define weather event profiles and identify their relevant characteristics
Notebook containing program code and data for the bachelor thesis of Julian Erath<br>
Submitted 08th of Mai 2023<br><br>


##### Supervisors:
Zhangziman Song, IBM Data Scientist for AI Applications<br>
Harini Srinivasan, IBM Manager for B2B DS & AI Applications <br>
Julia Wiegel, IBM Solutions Engineer & Meteorologist<br>

## Content

0. Introduction <br>
1. Imports, functions and options<br>
2. Data<br>
   1. Loading data<br>
   2. Inspect data
   3. Preparation
     - cleaning data
     - Removing NaN values with neighbor values
3. Regular clustering model
   1. Defining k number of clusters <br>
   2. Clustering algorithm <br>
   3. Evaluation metrices <br>
   4. Data visualisation and results<br>
4. Cascaded clustering model
   1. First step clustering
   2. Second step clustering
5. PCA clustering model
   1. Generate principal components 
   2. Analysis and results
   3. Inspect relevance of features
6. Seasons clustering model
   1. Defining k number of clusters <br>
   2. Clustering algorithm <br>
   3. Evaluation metrices <br>
   4. Data visualisation and results<br>
7. Create weather event profiles
   1. Map storm to storm2

---

## 1. Imports, functions and options<br>


In [ ]:
# imports

import pandas as pd
import numpy as np
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib import rcParams
import warnings
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
# Dataset
from sklearn import datasets
# Dimensionality reduction
from sklearn.decomposition import PCA
# Modeling
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN
#Feature Engineering / Standardization
from scipy.stats import zscore
from sklearn import preprocessing
# Evaluation metrices
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
import time
import os, psutil
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from resource import *
import datetime
warnings.filterwarnings("ignore")
rcParams['figure.figsize'] = 20,15
sns.set(style="darkgrid")

# functions
# evaluation 
def eval_metrics(X):
    silhouette_score_lst = []
    elbow_score_lst = []
    n_cluster = range(2, 20)
    for i in n_cluster:
        model = GaussianMixture(n_components = i, random_state = 42, reg_covar=1e-3)
        gmm_pred = model.fit_predict(X)
        elbow_score_lst.append(model.score(X))
        print(i, " cluster silhouette score: ", silhouette_score(X, gmm_pred))
        silhouette_score_lst.append(silhouette_score(X, gmm_pred))
    eval = pd.DataFrame(index=pd.Series(n_cluster, name='k')) 
    eval['elbow'] = elbow_score_lst
    eval['silhouette'] = silhouette_score_lst   
    return eval

def plot_eval_metrics(eval):
    f, axes = plt.subplots(1,2, figsize=[8,4], constrained_layout = True)
    axes = axes.ravel()

    eval['elbow'].plot(ax=axes[0], title="Elbow score")
    axes[0].set_xlabel('Number of clusters')
    axes[0].set_ylabel("Elbow score")

    eval['silhouette'].plot(ax=axes[1], title="Silhouette score")
    axes[1].set_xlabel('Number of clusters')
    axes[1].set_ylabel("Silhouette score")
    return f
    
def plot_cluster_feature(df, label_col):
    feature_show_lst = ['avg_temp','min_wet_bulb_temp','avg_dewpoint','avg_temp_change','avg_windspd','max_windgust',
    'avg_winddir','max_cumulative_precip','max_snow_density_6',
    'max_cumulative_snow','max_cumulative_ice','avg_pressure_change']

    f, axes = plt.subplots(4, 3, constrained_layout = True, figsize=[20,15])
    sns.set(style="darkgrid")
    for i in df[label_col].unique():
        df_cluster_distogram = df.loc[df[label_col] == i]
        axes = axes.ravel()
        num_label = df_cluster_distogram[label_col].value_counts()[i]
        for j, feature in enumerate(feature_show_lst):
            sns.distplot(df_cluster_distogram[feature], label=f'C{i};{num_label}', ax=axes[j])
    axes[0].set(title="Avg temperature in Kelvin", xlim=(230, 310), xlabel="")
    axes[0].legend(loc="best")
    axes[1].set(title="Min wet bulb temperature in Kelvin", xlim=(230, 310), xlabel="")
    axes[2].set(title="Avg dewpoint in Kelvin", xlim=(230, 310), xlabel="")
    axes[3].set(title="Avg temperature change in Kelvin", xlim=(-1, 1), xlabel="")
    axes[4].set(title="Avg wind speed in m/s", xlim=(0, 10), xlabel="")
    axes[5].set(title="Max windgust in m/s", xlim=(0, 30), xlabel="")
    axes[6].set(title="Avg winddirection in degrees", xlim=(0, 360), xlabel="")
    axes[7].set(title="Max cumulative precipitation in mm", xlim=(0, 100), ylim=(0, 0.1), xlabel="")
    axes[8].set(title="Max snow density (when snow exceeds 6.35mm in kg/m^3)", xlim=(0, 800), ylim=(0, 0.01), xlabel="")
    axes[9].set(title="Max cumulative snow accretion in mm", xlim=(0, 1000), ylim=(0, 0.009), xlabel="")
    axes[10].set(title="Max cumulative ice accretion in mm", xlim=(0, 10), ylim=(0, 0.8), xlabel="")
    axes[11].set(title="Avg pressure change in Pa",  xlim=(-200, 200), xlabel="")  
    return f
    

#Code mit Änderungen übernommen aus: https://matplotlib.org/stable/gallery/specialty_plots/radar_chart.html abgerufen am 19.04.2023 09:47 Uhr
def radar_factory(num_vars, frame='circle'):
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

    class RadarTransform(PolarAxes.PolarTransform):

        def transform_path_non_affine(self, path):
            # Paths with non-unit interpolation steps correspond to gridlines,
            # in which case we force interpolation (to defeat PolarTransform's
            # autoconversion to circular arcs).
            if path._interpolation_steps > 1:
                path = path.interpolated(num_vars)
            return Path(self.transform(path.vertices), path.codes)

    class RadarAxes(PolarAxes):

        name = 'radar'
        PolarTransform = RadarTransform

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k")
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                                    + self.transAxes)
                return {'polar': spine}
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta 

# Function to find all the local maxima and minima in the given array arr[]
# Code mit Änderungen übernommen aus https://www.geeksforgeeks.org/find-indices-of-all-local-maxima-and-local-minima-in-an-array/ abgerufen am 19.04.2023 09:47 Uhr
def findLocalMaximaMinima(n, arr):
    mx = []
    mn = []
    if(arr[0] > arr[1]):
        mx.append(0)
    elif(arr[0] < arr[1]):
        mn.append(0)
    for i in range(1, n-1):
        if(arr[i-1] > arr[i] < arr[i + 1]):
            mn.append(i)
        elif(arr[i-1] < arr[i] > arr[i + 1]):
            mx.append(i)
    if(arr[-1] > arr[-2]):
        mx.append(n-1)
    elif(arr[-1] < arr[-2]):
        mn.append(n-1)
    if(len(mx) > 0):
        return(mx)
    else:
        print("There are no points of"\
            " Local maxima.")
    if(len(mn) > 0):
        return(mn)
    else:
        print("There are no points"\
            " of Local minima.")

def assignStormIdHour(datetime_series, n_hours_after):
    """ Identify same storm happened at multiple hours and multiple locations
    and assign a list of storm id.
    """
    datetime_lst = list(datetime_series)
    index = datetime_series.index
    
    # Initialize the first storm id
    n = 0
    # Initialize storm id list with the first storm id
    storm_id_lst = [n]
    
    # This candidate will be assigned the same storm id if it is n_hours_after the last candidate in the datetime_lst
    for i in range(1, len(datetime_lst)):
        this = datetime_lst[i]
        last = datetime_lst[i-1]
        
        ## Define last candidate's neighbour
        # last candidate and it's right neighbour
        neighbour_dt_start = last
        neighbour_dt_end = last + datetime.timedelta(hours=n_hours_after)
        
        ## Assign storm id
        # if this candidate is in last candidate's neighbour hours, share the same storm id; else, assign new storm id
        if this not in pd.date_range(neighbour_dt_start, neighbour_dt_end, freq='H'):
            n += 1   
        # every date/loop need a storm id
        storm_id_lst.append(n)
    
    return pd.Series(storm_id_lst, index = index) 

def create_storm_table(df, id_col, agg_level):
    ''' Summarize storm information from id_col
    '''
    storm_sbs = df.groupby([id_col]+agg_level).agg({
        'valid_datetime':['min','max'],
        })
    
    storm_sbs.columns = ['storm_start_dh','storm_end_dh']
    storm_sbs = storm_sbs.reset_index()
    storm_sbs['duration'] = storm_sbs['storm_end_dh'] - storm_sbs['storm_start_dh']
    storm_sbs['duration_hour'] = storm_sbs['duration'].apply(lambda x: (x.total_seconds()/3600))
    storm_sbs = storm_sbs.rename(columns={id_col:'storm_id'})

    return storm_sbs

# measuring time and ressources
startTime = time.time()
startTimeCPU = time.process_time()

## 2. Data
- Loading data<br>
- Inspect data
- Preparation
  - cleaning data
  - Removing NaN values with neighbor values

In [ ]:
pd.options.display.max_columns = 999

In [ ]:
# Loading data
df = pd.read_csv('/Users/julianerath/Documents/Bachelor Thesis/Python/Analysis_per_region/Bancroft/feature_data_substation_Bancroft.csv')

#Set new index based on run_datetime so the data is chronologially sorted
df.sort_values(by='run_datetime', inplace = True, ignore_index=True)
df

In [ ]:
df.describe()

In [ ]:
# Calculating avg temp change and sin / cos of winddir
df["avg_temp_change"]=df["avg_temp"].diff()
df["min_temp_change"]=df["min_temp"].diff()
df["max_temp_change"]=df["max_temp"].diff()

df["avg_winddir_sin"]=np.sin(df["avg_winddir"])
df["avg_winddir_cos"]=np.cos(df["avg_winddir"])
df["min_winddir_sin"]=np.sin(df["min_winddir"])
df["min_winddir_cos"]=np.cos(df["min_winddir"])
df["max_winddir_sin"]=np.sin(df["max_winddir"])
df["max_winddir_cos"]=np.cos(df["max_winddir"])

df

In [ ]:
# Fill NaN with previous valid value or drop rows since enough data is available
# df = df.fillna(method='ffill')
# df = df.fillna(0)


# only use one temp feature or temp change feature
feature_lst = ['max_windgust','avg_temp','max_snow_density_6', 'max_cumulative_precip',
'max_cumulative_ice','max_cumulative_snow','avg_windspd','avg_winddir_sin','avg_winddir_cos']
# 'avg_dewpoint','avg_temp', 'min_wet_bulb_temp', 'avg_temp_change', 'avg_pressure_change

df = df.dropna(subset=feature_lst, how='any')
df

In [ ]:
#Moving average over a week
df['MA_max_windgust_week'] = df.max_windgust.rolling(168, min_periods=1).mean()
df['MA_min_wet_bulb_temp_week'] = df.min_wet_bulb_temp.rolling(168, min_periods=1).mean()
df['MA_max_snow_density_6_week'] = df.max_snow_density_6.rolling(168, min_periods=1).mean()
df['MA_max_cumulative_precip_week'] = df.max_cumulative_precip.rolling(168, min_periods=1).mean()
df['MA_max_cumulative_ice_week'] = df.max_cumulative_ice.rolling(168, min_periods=1).mean()
df['MA_max_cumulative_snow_week'] = df.max_cumulative_snow.rolling(168, min_periods=1).mean()
df['MA_avg_windspd_week'] = df.avg_windspd.rolling(168, min_periods=1).mean()
df['MA_avg_pressure_change_week'] = df.avg_pressure_change.rolling(168, min_periods=1).mean()
df['MA_avg_temp_week'] = df.avg_temp.rolling(168, min_periods=1).mean()
df['MA_avg_dewpoint_week'] = df.avg_dewpoint.rolling(168, min_periods=1).mean()
df['MA_avg_winddir_week'] = df.avg_winddir.rolling(168, min_periods=1).mean()
df['MA_avg_temp_change_week'] = df.avg_temp_change.rolling(168, min_periods=1).mean()

#Moving average over a month
df['MA_max_windgust_month'] = df.max_windgust.rolling(720, min_periods=1).mean()
df['MA_min_wet_bulb_temp_month'] = df.min_wet_bulb_temp.rolling(720, min_periods=1).mean()
df['MA_max_snow_density_6_month'] = df.max_snow_density_6.rolling(720, min_periods=1).mean()
df['MA_max_cumulative_precip_month'] = df.max_cumulative_precip.rolling(720, min_periods=1).mean()
df['MA_max_cumulative_ice_month'] = df.max_cumulative_ice.rolling(720, min_periods=1).mean()
df['MA_max_cumulative_snow_month'] = df.max_cumulative_snow.rolling(720, min_periods=1).mean()
df['MA_avg_windspd_month'] = df.avg_windspd.rolling(720, min_periods=1).mean()
df['MA_avg_pressure_change_month'] = df.avg_pressure_change.rolling(720, min_periods=1).mean()
df['MA_avg_temp_month'] = df.avg_temp.rolling(720, min_periods=1).mean()
df['MA_avg_dewpoint_month'] = df.avg_dewpoint.rolling(720, min_periods=1).mean()
df['MA_avg_winddir_month'] = df.avg_winddir.rolling(720, min_periods=1).mean()
df['MA_avg_temp_change_month'] = df.avg_temp_change.rolling(720, min_periods=1).mean()

In [ ]:
# Inspecting the data
fig, axes = plt.subplots(4, 3)

axes[0, 0].plot(df.run_datetime, df.avg_temp)
axes[0, 0].plot(df.run_datetime, df.MA_avg_temp_week)
axes[0, 0].plot(df.run_datetime, df.MA_avg_temp_month)
axes[0, 0].set_title('Bancroft avg_temp')
axes[0, 0].set_ylabel("Kelvin")
axes[0, 0].get_xaxis().set_visible(False)

axes[0, 1].plot(df.run_datetime, df.min_wet_bulb_temp)
axes[0, 1].plot(df.run_datetime, df.MA_min_wet_bulb_temp_week)
axes[0, 1].plot(df.run_datetime, df.MA_min_wet_bulb_temp_month)
axes[0, 1].set_title('Bancroft min_wet_bulb_temp')
axes[0, 1].set_ylabel("Kelvin")
axes[0, 1].get_xaxis().set_visible(False)

axes[0, 2].plot(df.run_datetime, df.avg_dewpoint)
axes[0, 2].plot(df.run_datetime, df.MA_avg_dewpoint_week)
axes[0, 2].plot(df.run_datetime, df.MA_avg_dewpoint_month)
axes[0, 2].set_title('Bancroft avg_dewpoint')
axes[0, 2].set_ylabel("Kelvin")
axes[0, 2].get_xaxis().set_visible(False)

axes[1, 0].plot(df.run_datetime, df.avg_temp_change)
axes[1, 0].plot(df.run_datetime, df.MA_avg_temp_change_week)
axes[1, 0].plot(df.run_datetime, df.MA_avg_temp_change_month)
axes[1, 0].set_title('Bancroft avg_temp_change')
axes[1, 0].set_ylabel("Kelvin")
axes[1, 0].get_xaxis().set_visible(False)

axes[1, 1].plot(df.run_datetime, df.avg_windspd)
axes[1, 1].plot(df.run_datetime, df.MA_avg_windspd_week)
axes[1, 1].plot(df.run_datetime, df.MA_avg_windspd_month)
axes[1, 1].set_title('Bancroft avg_windspd')
axes[1, 1].set_ylabel("m/s")
axes[1, 1].get_xaxis().set_visible(False)

axes[1, 2].plot(df.run_datetime, df.max_windgust)
axes[1, 2].plot(df.run_datetime, df.MA_max_windgust_week)
axes[1, 2].plot(df.run_datetime, df.MA_max_windgust_month)
axes[1, 2].set_title('Bancroft max_windgust')
axes[1, 2].set_ylabel("m/s")
axes[1, 2].get_xaxis().set_visible(False)

axes[2, 0].plot(df.run_datetime, df.avg_winddir)
axes[2, 0].plot(df.run_datetime, df.MA_avg_winddir_week)
axes[2, 0].plot(df.run_datetime, df.MA_avg_winddir_month)
axes[2, 0].set_title('Bancroft avg_winddir')
axes[2, 0].set_ylabel("degrees")
axes[2, 0].get_xaxis().set_visible(False)

axes[2, 1].plot(df.run_datetime, df.max_cumulative_precip)
axes[2, 1].plot(df.run_datetime, df.MA_max_cumulative_precip_week)
axes[2, 1].plot(df.run_datetime, df.MA_max_cumulative_precip_month)
axes[2, 1].set_title('Bancroft max_cumulative_precip')
axes[2, 1].set_ylabel("mm")
axes[2, 1].get_xaxis().set_visible(False)

axes[2, 2].plot(df.run_datetime, df.max_snow_density_6)
axes[2, 2].plot(df.run_datetime, df.MA_max_snow_density_6_week)
axes[2, 2].plot(df.run_datetime, df.MA_max_snow_density_6_month)
axes[2, 2].set_title('Bancroft max_snow_density_6')
axes[2, 2].set_ylabel("kg/m^3")
axes[2, 2].get_xaxis().set_visible(False)

axes[3, 0].plot(df.run_datetime, df.max_cumulative_snow)
axes[3, 0].plot(df.run_datetime, df.MA_max_cumulative_snow_week)
axes[3, 0].plot(df.run_datetime, df.MA_max_cumulative_snow_month)
axes[3, 0].set_title('Bancroft max_cumulative_snow')
axes[3, 0].set_ylabel("mm")
axes[3, 0].get_xaxis().set_visible(False)

axes[3, 1].plot(df.run_datetime, df.max_cumulative_ice)
axes[3, 1].plot(df.run_datetime, df.MA_max_cumulative_ice_week)
axes[3, 1].plot(df.run_datetime, df.MA_max_cumulative_ice_month)
axes[3, 1].set_title('Bancroft max_cumulative_ice')
axes[3, 1].set_ylabel("mm")
axes[3, 1].get_xaxis().set_visible(False)

axes[3, 2].plot(df.run_datetime, df.avg_pressure_change)
axes[3, 2].plot(df.run_datetime, df.MA_avg_pressure_change_week)
axes[3, 2].plot(df.run_datetime, df.MA_avg_pressure_change_month)
axes[3, 2].set_title('Bancroft avg_pressure_change')
axes[3, 2].set_ylabel("Pa")
axes[3, 2].get_xaxis().set_visible(False)

plt.suptitle('Bancroft weather parameters time series')
plt.legend(loc="best")
plt.show()
plt.savefig('Bancroft weather parameters time series.png')


In [ ]:
# df = pd.read_csv('/Users/julianerath/Documents/Bachelor Thesis/Python/Data/ERA5_features_20150715-20221230/feature_data_substation_Bancroft.csv', parse_dates=['valid_datetime'], index_col=0)
df_normalized = df[['max_windgust','max_snow_density_6', 'max_cumulative_precip',
'max_cumulative_ice','max_cumulative_snow','avg_windspd','avg_winddir_sin','avg_winddir_cos',
'avg_dewpoint','avg_temp', 'min_wet_bulb_temp', 'avg_temp_change', 'avg_pressure_change']]

df_normalized = pd.DataFrame(preprocessing.StandardScaler().fit_transform(df_normalized),columns=df_normalized.columns)



df = df.dropna(subset=feature_lst, how='any')
X = pd.DataFrame(preprocessing.StandardScaler().fit_transform(df[feature_lst]),columns=feature_lst)


## 3. Regular clustering model

### 3.1. Defining k number of clusters <br>


In [ ]:
startTime_regular_clustering_model = time.time()
startTimeCPU_regular_clustering_model = time.process_time()

In [ ]:
eval_regular_clustering_model = eval_metrics(X)
eval_regular_clustering_model.to_csv('eval_regular_clustering_model.csv')
eval_regular_clustering_model = pd.read_csv('eval_regular_clustering_model.csv', index_col=0)
plot_eval_metrics(eval_regular_clustering_model).savefig('Bancroft elbow and silhouette score regular clustering model.png')

In [ ]:
all_maxima = findLocalMaximaMinima(18, eval_regular_clustering_model.silhouette.values.tolist())
second_maximum = all_maxima[1]
k0 = second_maximum+2
print("The optimal number of k clusters for the regular clustering model is: ", k0)

###   3.2. Clustering algorithm <br>
###   3.3. Evaluation metrices <br>


In [ ]:
gmm_regular_clustering_model = GaussianMixture(n_components = k0, random_state = 42, reg_covar=1e-3)
labels_regular_clustering_mode = gmm_regular_clustering_model.fit_predict(X)
print("Model score: ", gmm_regular_clustering_model.score(X), "\nSilhouette score: ", silhouette_score(X, labels_regular_clustering_mode)) 

df['label0'] = pd.Series(labels_regular_clustering_mode, index=df.index)
df_normalized['label0'] = pd.Series(labels_regular_clustering_mode, index=df.index)

display(df['label0'].value_counts())

### 3.4. Data visualisation and results<br>

In [ ]:
df_radar_charts_regular_clustering_model = df_normalized[['max_windgust', 'min_wet_bulb_temp', 'max_snow_density_6', 
'max_cumulative_precip', 'max_cumulative_ice', 'max_cumulative_snow', 'avg_windspd', 'avg_pressure_change', 'avg_temp', 
'avg_dewpoint', 'avg_winddir_sin','avg_winddir_cos', 'avg_temp_change','label0']]

df_cluster_groups_mean_regular_clustering_model = df_radar_charts_regular_clustering_model.groupby(['label0'],as_index=False).mean()
df_cluster_groups_mean_regular_clustering_model = df_cluster_groups_mean_regular_clustering_model.drop("label0", axis=1)

df_cluster_groups_max_regular_clustering_model = df_radar_charts_regular_clustering_model.groupby(['label0'],as_index=False).max()
df_cluster_groups_max_regular_clustering_model = df_cluster_groups_max_regular_clustering_model.drop('label0', axis=1)

df_cluster_groups_min_regular_clustering_model = df_radar_charts_regular_clustering_model.groupby(['label0'],as_index=False).min()
df_cluster_groups_min_regular_clustering_model = df_cluster_groups_min_regular_clustering_model.drop('label0', axis=1)

df_cluster_groups_median_regular_clustering_model = df_radar_charts_regular_clustering_model.groupby(['label0'],as_index=False).median()
df_cluster_groups_median_regular_clustering_model = df_cluster_groups_median_regular_clustering_model.drop('label0', axis=1)

In [ ]:
def example_data():
    for i in range(0, 6):
                data = [
                    ['max_windgust', 'min_wet_bulb_temp', 'max_snow_density_6', 'max_cumulative_precip', 
                    'max_cumulative_ice', 'max_cumulative_snow', 'avg_windspd', 'avg_pressure_change', 
                    'avg_temp', 'avg_dewpoint', 'avg_winddir_sin','avg_winddir_cos', 'avg_temp_change'],
                    ('Mean', [
                        df_cluster_groups_mean_regular_clustering_model.iloc[0],
                        df_cluster_groups_mean_regular_clustering_model.iloc[1],
                        df_cluster_groups_mean_regular_clustering_model.iloc[2],
                        df_cluster_groups_mean_regular_clustering_model.iloc[3],
                        df_cluster_groups_mean_regular_clustering_model.iloc[4],
                        df_cluster_groups_mean_regular_clustering_model.iloc[5],
                    ]),
                    ('Max', [
                        df_cluster_groups_max_regular_clustering_model.iloc[0],
                        df_cluster_groups_max_regular_clustering_model.iloc[1],
                        df_cluster_groups_max_regular_clustering_model.iloc[2],
                        df_cluster_groups_max_regular_clustering_model.iloc[3],
                        df_cluster_groups_max_regular_clustering_model.iloc[4],
                        df_cluster_groups_max_regular_clustering_model.iloc[5],
                        ]),
                    ('Min', [
                        df_cluster_groups_min_regular_clustering_model.iloc[0],
                        df_cluster_groups_min_regular_clustering_model.iloc[1],
                        df_cluster_groups_min_regular_clustering_model.iloc[2],
                        df_cluster_groups_min_regular_clustering_model.iloc[3],
                        df_cluster_groups_min_regular_clustering_model.iloc[4],
                        df_cluster_groups_min_regular_clustering_model.iloc[5],
                        ]),
                    ('Median', [
                        df_cluster_groups_median_regular_clustering_model.iloc[0],
                        df_cluster_groups_median_regular_clustering_model.iloc[1],
                        df_cluster_groups_median_regular_clustering_model.iloc[2],
                        df_cluster_groups_median_regular_clustering_model.iloc[3],
                        df_cluster_groups_median_regular_clustering_model.iloc[4],
                        df_cluster_groups_median_regular_clustering_model.iloc[5],
                        ])
                ]
    return data


if __name__ == '__main__':
    theta = radar_factory(13, frame='polygon')

    data = example_data()
    spoke_labels = data.pop(0)

    fig, axs = plt.subplots(figsize=(14, 9), nrows=2, ncols=2,
                            subplot_kw=dict(projection='radar'))
    fig.subplots_adjust(wspace=0.20, hspace=0.5, top=0.85, bottom=0.05)

    colors = ['b', 'r', 'g', 'm', 'y', 'w']
    # Plot the four cases from the example data on separate axes
    for ax, (title, case_data) in zip(axs.flat, data):
        ax.set_yticks([-10, -5, 0, 5, 10], ["-10","-5","0","5","10"], color="grey", size=7)
        ax.set_ylim(-10,10)
        ax.set_rgrids([-10, -5, 0, 5, 10])
        ax.set_title(title, weight='bold', size='medium', position=(0.5, 1.1),
                     horizontalalignment='center', verticalalignment='center')
        for d, color in zip(case_data, colors):
            ax.plot(theta, d, color=color)
            ax.fill(theta, d, facecolor=color, alpha=0.25, label='_nolegend_')
        ax.set_varlabels(spoke_labels)

    # add legend relative to top-left plot
    cluster_legend_names = {}
    for i in range(0, k0):
        cluster_legend_names.update({"num_label_%s" % (i): "C%s %s" % (i, df_radar_charts_regular_clustering_model['label0'].value_counts()[i])})

    labels = tuple(cluster_legend_names.values())
    legend = axs[0, 0].legend(labels, loc=(1.1, 1),
                              labelspacing=0.1, fontsize='small')

    fig.text(0.5, 0.965, 'Bancroft weather clusters radar charts regular clustering model',
             horizontalalignment='center', color='black', weight='bold',
             size='large')


    plt.show()
    fig.savefig('Bancroft weather clusters radar charts regular clustering model.png')

In [ ]:
print(df[feature_lst+['label0']].groupby('label0').describe())
df[feature_lst+['label0']].groupby('label0').describe().to_csv("Regular clustering model statistics Bancroft GaussianMixture.csv")

In [ ]:
data = df.dropna(subset=['label0'], how='any').sort_values('label0')
data = data.query("label0 in [0,1,2,3]")
plot_cluster_feature(data, 'label0').savefig(f'Bancroft weather clusters distograms regular method cluster 0 to 3.png')


In [ ]:
data = df.dropna(subset=['label0'], how='any').sort_values('label0')
data = data.query("label0 in [4,5,6,7]")
plot_cluster_feature(data, 'label0').savefig(f'Bancroft weather clusters distograms regular method cluster 4 to 7.png')


In [ ]:
endTime_regular_clustering_model = time.time()
elapsedTime_regular_clustering_model = endTime_regular_clustering_model - startTime_regular_clustering_model
print("Execution time: ", elapsedTime_regular_clustering_model, " seconds")

endTimeCPU_regular_clustering_model = time.process_time()
res_regular_clustering_model =  endTimeCPU_regular_clustering_model - startTimeCPU_regular_clustering_model
print("CPU Execution time: ", res_regular_clustering_model, " seconds")


process = psutil.Process(os.getpid())
print("used non-swapped physical memory (in kiloBytes)", process.memory_info().rss, " kiloBytes")  

print(getrusage(RUSAGE_SELF))
print(getrusage(RUSAGE_SELF).ru_maxrss)

## 4. Cascaded clustering model


In [ ]:
startTime_cascaded_clustering_model = time.time()
startTimeCPU_cascaded_clustering_model = time.process_time()

### 4.1. First step clustering

In [ ]:
eval1 = eval_metrics(X)
eval1.to_csv('eval1.csv')
eval1 = pd.read_csv('eval1.csv', index_col=0)
plot_eval_metrics(eval1).savefig('Bancroft elbow and silhouette score cascaded clustering model step one.png')

In [ ]:
k1 = eval1['silhouette'].idxmax()
print("The optimal number of k clusters for first step of cascaded clustering is: ", k1)


In [ ]:
gmm = GaussianMixture(n_components = k1, random_state = 42, reg_covar=1e-3)
labels = gmm.fit_predict(X)

print("Model score: ", gmm.score(X), "\nSilhouette score: ", silhouette_score(X, labels))
df['label1'] = pd.Series(labels, index=df.index)
storm_cluster = df['label1'].value_counts().idxmin()
df_normalized['label1'] = pd.Series(labels, index=df.index)
df_sub = df.query(f'label1=={storm_cluster}')

display(df['label1'].value_counts())

In [ ]:
df_sub = df.query(f'label1=={storm_cluster}')
df_sub.shape

### 4.2. Second step clustering

In [ ]:
eval2 = eval_metrics(df_sub[feature_lst])
plot_eval_metrics(eval2).savefig('Bancroft elbow and silhouette score cascaded clustering model step two.png')

In [ ]:
# k2 = eval2['silhouette'].idxmax()
# print("The optimal number of k clusters for the second step of cascaded clustering is: ", k2)


all_maxima = findLocalMaximaMinima(18, eval2.silhouette.values.tolist())
second_maximum = all_maxima[1]
k2 = second_maximum+2
print("The optimal number of k clusters for the regular clustering model is: ", k2)

In [ ]:
gmm = GaussianMixture(n_components = k2, random_state = 42, reg_covar=1e-3)
label2 = gmm.fit_predict(df_sub[feature_lst])

print("Model score: ", gmm.score(df_sub[feature_lst]), "\nSilhouette score: ", silhouette_score(df_sub[feature_lst], label2))
df['label2'] = pd.Series(label2, index=df_sub.index)
df_normalized['label2'] = pd.Series(label2, index=df_sub.index)
df['label2'] = df['label2'].dropna().map(int)
display(df['label2'].value_counts())

In [ ]:
df_radar_charts = df_normalized[['max_windgust', 'min_wet_bulb_temp', 'max_snow_density_6', 'max_cumulative_precip', 
                    'max_cumulative_ice', 'max_cumulative_snow', 'avg_windspd', 'avg_pressure_change', 
                    'avg_temp', 'avg_dewpoint', 'avg_winddir_sin','avg_winddir_cos', 'avg_temp_change','label1']]

df_cluster_groups_mean = df_radar_charts.groupby(['label1'],as_index=False).mean()
df_cluster_groups_mean = df_cluster_groups_mean.drop("label1", axis=1)

df_cluster_groups_max = df_radar_charts.groupby(['label1'],as_index=False).max()
df_cluster_groups_max = df_cluster_groups_max.drop('label1', axis=1)

df_cluster_groups_min = df_radar_charts.groupby(['label1'],as_index=False).min()
df_cluster_groups_min = df_cluster_groups_min.drop('label1', axis=1)

df_cluster_groups_median = df_radar_charts.groupby(['label1'],as_index=False).median()
df_cluster_groups_median = df_cluster_groups_median.drop('label1', axis=1)

In [ ]:
def example_data():
    for i in range(0, 6):
                data = [
                    ['max_windgust', 'min_wet_bulb_temp', 'max_snow_density_6', 'max_cumulative_precip', 
                    'max_cumulative_ice', 'max_cumulative_snow', 'avg_windspd', 'avg_pressure_change', 
                    'avg_temp', 'avg_dewpoint', 'avg_winddir_sin','avg_winddir_cos', 'avg_temp_change'],
                    ('Mean', [
                        df_cluster_groups_mean.iloc[0],
                        df_cluster_groups_mean.iloc[1],
                    ]),
                    ('Max', [
                        df_cluster_groups_max.iloc[0],
                        df_cluster_groups_max.iloc[1],
                        ]),
                    ('Min', [
                        df_cluster_groups_min.iloc[0],
                        df_cluster_groups_min.iloc[1],
                        ]),
                    ('Median', [
                        df_cluster_groups_median.iloc[0],
                        df_cluster_groups_median.iloc[1],
                        ])
                ]
    return data


if __name__ == '__main__':
    theta = radar_factory(13, frame='polygon')

    data = example_data()
    spoke_labels = data.pop(0)

    fig, axs = plt.subplots(figsize=(14, 9), nrows=2, ncols=2,
                            subplot_kw=dict(projection='radar'))
    fig.subplots_adjust(wspace=0.20, hspace=0.5, top=0.85, bottom=0.05)

    colors = ['b', 'r', 'g', 'm', 'y', 'w']
    # Plot the four cases from the example data on separate axes
    for ax, (title, case_data) in zip(axs.flat, data):
        ax.set_yticks([-10, -5, 0, 5, 10], ["-10","-5","0","5","10"], color="grey", size=7)
        ax.set_ylim(-10,10)
        ax.set_rgrids([-10, -5, 0, 5, 10])
        ax.set_title(title, weight='bold', size='medium', position=(0.5, 1.1),
                     horizontalalignment='center', verticalalignment='center')
        for d, color in zip(case_data, colors):
            ax.plot(theta, d, color=color)
            ax.fill(theta, d, facecolor=color, alpha=0.25, label='_nolegend_')
        ax.set_varlabels(spoke_labels)

    # add legend relative to top-left plot
    cluster_legend_names = {}
    for i in range(0, k1):
        cluster_legend_names.update({"num_label_%s" % (i): "C%s %s" % (i, df_radar_charts['label1'].value_counts()[i])})

    labels = tuple(cluster_legend_names.values())
    legend = axs[0, 0].legend(labels, loc=(1.1, 1),
                              labelspacing=0.1, fontsize='small')

    fig.text(0.5, 0.965, 'Bancroft weather clusters radar charts',
             horizontalalignment='center', color='black', weight='bold',
             size='large')


    plt.show()
    fig.savefig('Bancroft weather clusters radar charts cascaded clustering first step.png')

In [ ]:
print(df[feature_lst+['label1']].groupby('label1').describe())
df[feature_lst+['label1']].groupby('label1').describe().to_csv("Cascaced clustering model first step statistics Bancroft GaussianMixture.csv")

In [ ]:
data = df.dropna(subset=['label1'], how='any').sort_values('label1')
data = data.query("label1 in [0,1,2,3,4,5,6,7,8,9,10]")
plot_cluster_feature(data, 'label1').savefig(f'Bancroft weather clusters distograms cascaded method step one.png')


In [ ]:
df_radar_charts = df_normalized[['max_windgust', 'min_wet_bulb_temp', 'max_snow_density_6', 'max_cumulative_precip', 
                    'max_cumulative_ice', 'max_cumulative_snow', 'avg_windspd', 'avg_pressure_change', 
                    'avg_temp', 'avg_dewpoint', 'avg_winddir_sin','avg_winddir_cos', 'avg_temp_change','label2']]

df_cluster_groups_mean = df_radar_charts.groupby(['label2'],as_index=False).mean()
df_cluster_groups_mean = df_cluster_groups_mean.drop("label2", axis=1)

df_cluster_groups_max = df_radar_charts.groupby(['label2'],as_index=False).max()
df_cluster_groups_max = df_cluster_groups_max.drop('label2', axis=1)

df_cluster_groups_min = df_radar_charts.groupby(['label2'],as_index=False).min()
df_cluster_groups_min = df_cluster_groups_min.drop('label2', axis=1)

df_cluster_groups_median = df_radar_charts.groupby(['label2'],as_index=False).median()
df_cluster_groups_median = df_cluster_groups_median.drop('label2', axis=1)

In [ ]:
def example_data():
    for i in range(0, 6):
                data = [
                    ['max_windgust', 'min_wet_bulb_temp', 'max_snow_density_6', 'max_cumulative_precip', 
                    'max_cumulative_ice', 'max_cumulative_snow', 'avg_windspd', 'avg_pressure_change', 
                    'avg_temp', 'avg_dewpoint', 'avg_winddir_sin','avg_winddir_cos', 'avg_temp_change'],
                    ('Mean', [
                        df_cluster_groups_mean.iloc[0],
                        df_cluster_groups_mean.iloc[1],
                        df_cluster_groups_mean.iloc[2],
                        df_cluster_groups_mean.iloc[3],
                        df_cluster_groups_mean.iloc[4],
                        df_cluster_groups_mean.iloc[5],
                    ]),
                    ('Max', [
                        df_cluster_groups_max.iloc[0],
                        df_cluster_groups_max.iloc[1],
                        df_cluster_groups_max.iloc[2],
                        df_cluster_groups_max.iloc[3],
                        df_cluster_groups_max.iloc[4],
                        df_cluster_groups_max.iloc[5],
                        ]),
                    ('Min', [
                        df_cluster_groups_min.iloc[0],
                        df_cluster_groups_min.iloc[1],
                        df_cluster_groups_min.iloc[2],
                        df_cluster_groups_min.iloc[3],
                        df_cluster_groups_min.iloc[4],
                        df_cluster_groups_min.iloc[5],
                        ]),
                    ('Median', [
                        df_cluster_groups_median.iloc[0],
                        df_cluster_groups_median.iloc[1],
                        df_cluster_groups_median.iloc[2],
                        df_cluster_groups_median.iloc[3],
                        df_cluster_groups_median.iloc[4],
                        df_cluster_groups_median.iloc[5],
                        ])
                ]
    return data


if __name__ == '__main__':
    theta = radar_factory(13, frame='polygon')

    data = example_data()
    spoke_labels = data.pop(0)

    fig, axs = plt.subplots(figsize=(14, 9), nrows=2, ncols=2,
                            subplot_kw=dict(projection='radar'))
    fig.subplots_adjust(wspace=0.20, hspace=0.5, top=0.85, bottom=0.05)

    colors = ['b', 'r', 'g', 'm', 'y', 'w']
    # Plot the four cases from the example data on separate axes
    for ax, (title, case_data) in zip(axs.flat, data):
        ax.set_yticks([-10, -5, 0, 5, 10], ["-10","-5","0","5","10"], color="grey", size=7)
        ax.set_ylim(-10,10)
        ax.set_rgrids([-10, -5, 0, 5, 10])
        ax.set_title(title, weight='bold', size='medium', position=(0.5, 1.1),
                     horizontalalignment='center', verticalalignment='center')
        for d, color in zip(case_data, colors):
            ax.plot(theta, d, color=color)
            ax.fill(theta, d, facecolor=color, alpha=0.25, label='_nolegend_')
        ax.set_varlabels(spoke_labels)

    # add legend relative to top-left plot
    cluster_legend_names = {}
    for i in range(0, k2):
        cluster_legend_names.update({"num_label_%s" % (i): "C%s %s" % (i, df_radar_charts['label2'].value_counts()[i])})

    labels = tuple(cluster_legend_names.values())
    legend = axs[0, 0].legend(labels, loc=(1.1, 1),
                              labelspacing=0.1, fontsize='small')

    fig.text(0.5, 0.965, 'Bancroft weather clusters radar charts',
             horizontalalignment='center', color='black', weight='bold',
             size='large')


    plt.show()
    fig.savefig('Bancroft weather clusters radar charts cascaded clustering second step.png')

In [ ]:
print(df[feature_lst+['label2']].groupby('label2').describe())
df[feature_lst+['label2']].groupby('label2').describe().to_csv("Cascaced clustering model second step statistics Bancroft GaussianMixture.csv")

In [ ]:
data = df.dropna(subset=['label2'], how='any').sort_values('label2')
data = data.query("label2 in [0,1,2,3]")
plot_cluster_feature(data, 'label2').savefig(f'Bancroft weather clusters distograms cascaded method step two (clusters 0 to 3).png')

In [ ]:
data = df.dropna(subset=['label2'], how='any').sort_values('label2')
data = data.query("label2 in [4,5,6]")
plot_cluster_feature(data, 'label2').savefig(f'Bancroft weather clusters distograms cascaded method step two (clusters 4 to 6).png')

In [ ]:
endTime_cascaded_clustering_model = time.time()
elapsedTime_cascaded_clustering_model = endTime_cascaded_clustering_model - startTime_cascaded_clustering_model
print("Execution time: ", elapsedTime_cascaded_clustering_model, " seconds")

endTimeCPU_cascaded_clustering_model = time.process_time()
res_cascaded_clustering_model =  endTimeCPU_cascaded_clustering_model - startTimeCPU_cascaded_clustering_model
print("CPU Execution time: ", res_cascaded_clustering_model, " seconds")


process = psutil.Process(os.getpid())
print("used non-swapped physical memory (in kiloBytes)", process.memory_info().rss, " kiloBytes")  

print(getrusage(RUSAGE_SELF))
print(getrusage(RUSAGE_SELF).ru_maxrss)

## 5. PCA clustering model
### 5.1. Generate principal components 


In [ ]:
startTime_pca_clustering_model = time.time()
startTimeCPU_pca_clustering_model = time.process_time()

In [ ]:
pca = PCA(n_components=3)
pca_df_cluster = pca.fit_transform(X)

principal_components_df = pd.DataFrame(data = pca_df_cluster, columns = ['principal component 0', 'principal component 1', 'principal component 2'])
principal_components_df.to_csv("principal_components_df_Bancroft.csv")
principal_components_df

### 5.2. Analysis and results


In [ ]:
eval_pca = eval_metrics(pca_df_cluster)
plot_eval_metrics(eval_pca).savefig('Bancroft elbow and silhouette score pca clustering model.png')

In [ ]:
all_maxima = findLocalMaximaMinima(18, eval_pca.silhouette.values.tolist())
second_maximum = all_maxima[1]
k3 = second_maximum+2
print("The optimal number of k clusters for the pca model is: ", k3)

In [ ]:
gmm_pca = GaussianMixture(n_components = k3, random_state = 42, reg_covar=1e-3)
labels_pca = gmm_pca.fit_predict(principal_components_df)

print("Model score: ", gmm_pca.score(principal_components_df), "\nSilhouette score: ", silhouette_score(principal_components_df, labels_pca))
df['label3'] = pd.Series(labels_pca, index=df.index)
df_normalized['label3'] = pd.Series(labels_pca, index=df.index)
principal_components_df['label3'] = pd.Series(labels_pca, index=df.index)


display(df['label3'].value_counts())

In [ ]:
principal_components_df = principal_components_df.dropna()
principal_components_df['label3'] = principal_components_df['label3'].astype({'label3':'int'})

In [ ]:
principal_components_df_mean = principal_components_df.groupby(['label3'],as_index=False).mean()
principal_components_df_mean = principal_components_df_mean.drop('label3', axis=1)

principal_components_df_max = principal_components_df.groupby(['label3'],as_index=False)["principal component 0", "principal component 1", "principal component 2"].max()
principal_components_df_max = principal_components_df_max.drop('label3', axis=1)

principal_components_df_min = principal_components_df.groupby(['label3'],as_index=False)["principal component 0", "principal component 1", "principal component 2"].min()
principal_components_df_min = principal_components_df_min.drop('label3', axis=1)

principal_components_df_median = principal_components_df.groupby(['label3'],as_index=False)["principal component 0", "principal component 1", "principal component 2"].median()
principal_components_df_median = principal_components_df_median.drop('label3', axis=1)


def example_data():
    for i in range(0, 6):
                data = [
        ["principal component 0", "principal component 1", "principal component 2"],
        ('Mean', [
            principal_components_df_mean.iloc[0],
            principal_components_df_mean.iloc[1],
            principal_components_df_mean.iloc[2],
            principal_components_df_mean.iloc[3],
        ]),
        ('Max', [
            principal_components_df_max.iloc[0],
            principal_components_df_max.iloc[1],
            principal_components_df_max.iloc[2],
            principal_components_df_max.iloc[3],
            ]),
        ('Min', [
            principal_components_df_min.iloc[0],
            principal_components_df_min.iloc[1],
            principal_components_df_min.iloc[2],
            principal_components_df_min.iloc[3],
            ]),
        ('Median', [
            principal_components_df_median.iloc[0],
            principal_components_df_median.iloc[1],
            principal_components_df_median.iloc[2],
            principal_components_df_median.iloc[3],
            ])
    ]
    return data


if __name__ == '__main__':
    theta = radar_factory(3, frame='polygon')

    data = example_data()
    spoke_labels = data.pop(0)

    fig, axs = plt.subplots(figsize=(14, 9), nrows=2, ncols=2,
                            subplot_kw=dict(projection='radar'))
    fig.subplots_adjust(wspace=0.20, hspace=0.5, top=0.85, bottom=0.05)

    colors = ['b', 'r', 'g', 'm', 'y', 'w']
    # Plot the four cases from the example data on separate axes
    for ax, (title, case_data) in zip(axs.flat, data):
        ax.set_yticks([-10, -5, 0, 5, 10], ["-10","-5","0","5","10"], color="grey", size=7)
        ax.set_ylim(-10,10)
        ax.set_rgrids([-10, -5, 0, 5, 10])
        ax.set_title(title, weight='bold', size='medium', position=(0.5, 1.1),
                     horizontalalignment='center', verticalalignment='center')
        for d, color in zip(case_data, colors):
            ax.plot(theta, d, color=color)
            ax.fill(theta, d, facecolor=color, alpha=0.25, label='_nolegend_')
        ax.set_varlabels(spoke_labels)

    # add legend relative to top-left plot
    cluster_legend_names = {}
    for i in range(0, k3):
        cluster_legend_names.update({"num_label_%s" % (i): "C%s %s" % (i, principal_components_df['label3'].value_counts()[i])})

    labels = tuple(cluster_legend_names.values())
    legend = axs[0, 0].legend(labels, loc=(1.1, 1),
                              labelspacing=0.1, fontsize='small')

    fig.text(0.5, 0.965, 'Bancroft weather clusters radar charts',
             horizontalalignment='center', color='black', weight='bold',
             size='large')


    plt.show()
    fig.savefig('Bancroft weather clusters radar charts PCA.png')

In [ ]:
f, axes = plt.subplots(1, 3, constrained_layout = True, figsize=[12,4])
sns.set(style="darkgrid")
feature_lst_pca = ['principal component 0','principal component 1','principal component 2']

for i in principal_components_df['label3'].unique():
    df_cluster_distogram = principal_components_df.loc[principal_components_df['label3'] == i]
    axes = axes.ravel()
    num_label = df_cluster_distogram['label3'].value_counts()[i]
    for j, feature in enumerate(feature_lst_pca):
        sns.distplot(df_cluster_distogram[feature], label=f'C{i};{num_label}', ax=axes[j])
    axes[0].legend(loc="best")

f.savefig(f'Bancroft weather clusters distograms pca method.png')


### 5.3. Inspect relevance of features

In [ ]:
print(pca_df_cluster)
print(abs( pca.components_ ))
pca_feature_relevance = pd.DataFrame(abs( pca.components_ ))
pca_feature_relevance.to_csv("pca_feature_relevance_Bancroft.csv")
pca_feature_relevance

In [ ]:
trace1 = go.Scatter3d(
    x=pca_df_cluster[:,0],
    y = pca_df_cluster[:,1],
    z = pca_df_cluster[:,2],
    mode='markers',
    marker=dict(
        size=3,
        color= principal_components_df['label3'],                
        opacity=1
)

)

dc_1 = go.Scatter3d( x = [0,pca.components_.T[0][0]],
                     y = [0,pca.components_.T[0][1]],
                     z = [0,pca.components_.T[0][2]],
                     marker = dict( size = 1,
                                    color = "rgb(84,48,5)"),
                     line = dict( color = "red",
                                width = 6),
                     name = "Var1"
                     )
dc_2 = go.Scatter3d( x = [0,pca.components_.T[1][0]],
                   y = [0,pca.components_.T[1][1]],
                   z = [0,pca.components_.T[1][2]],
                   marker = dict( size = 1,
                                  color = "rgb(84,48,5)"),
                   line = dict( color = "green",
                                width = 6),
                   name = "Var2"
                 )
dc_3 = go.Scatter3d( x = [0,pca.components_.T[2][0]],
                     y = [0,pca.components_.T[2][1]],
                     z = [0,pca.components_.T[2][2]],
                     marker = dict( size = 1,
                                  color = "rgb(84,48,5)"),
                     line = dict( color = "blue",
                                width = 6),
                     name = "Var3"
                 ) 
dc_4 = go.Scatter3d( x = [0,pca.components_.T[3][0]],
                     y = [0,pca.components_.T[3][1]],
                     z = [0,pca.components_.T[3][2]],
                     marker = dict( size = 1,
                                  color = "rgb(84,48,5)"),
                     line = dict( color = "yellow",
                                width = 6),
                     name = "Var4"
                   )


data = [trace1,dc_1,dc_2,dc_3,dc_4]
layout = go.Layout(
    xaxis=dict(
        title='PC1',
        titlefont=dict(
           family='Courier New, monospace',
           size=18,
           color='#7f7f7f'
       )
   )
)
fig = go.Figure(data=data, layout=layout)
pyo.iplot(data, filename = 'basic-line')


#https://stackoverflow.com/questions/48385273/3d-biplot-in-plotly-in-python

In [ ]:
fig = px.scatter_3d(principal_components_df, x='principal component 0', y='principal component 1', z='principal component 2',
                    color='label3', symbol='label3', size_max=1, opacity=0.7)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_traces(marker=dict(size=2),
                  selector=dict(mode='markers'))

fig.show()


In [ ]:
endTime_pca_clustering_model = time.time()
elapsedTime_pca_clustering_model = endTime_pca_clustering_model - startTime_pca_clustering_model
print("Execution time: ", elapsedTime_pca_clustering_model, " seconds")

endTimeCPU_pca_clustering_model = time.process_time()
res_pca_clustering_model =  endTimeCPU_pca_clustering_model - startTimeCPU_pca_clustering_model
print("CPU Execution time: ", res_pca_clustering_model, " seconds")


process = psutil.Process(os.getpid())
print("used non-swapped physical memory (in kiloBytes)", process.memory_info().rss, " kiloBytes")  

print(getrusage(RUSAGE_SELF))
print(getrusage(RUSAGE_SELF).ru_maxrss)

## 7. Create weather event profiles

In [ ]:
# Naming the weather clusters
df['wep'] = df['label2']
df['wep'] = df['wep'].fillna('Blue sky day')
df['wep'].mask(df['wep'] == 0 ,'Moderate snowfall', inplace=True)
df['wep'].mask(df['wep'] == 1 ,'Heavy snowfall with accumulated snow', inplace=True)
df['wep'].mask(df['wep'] == 2 ,'Storm with freezing rain / Heavy snow- and icestorm', inplace=True)
df['wep'].mask(df['wep'] == 3 ,'Moderate rain', inplace=True)
df['wep'].mask(df['wep'] == 4 ,'Snowstorm with high precipitation', inplace=True)
df['wep'].mask(df['wep'] == 5 ,'Mild snowfall', inplace=True)
df['wep'].mask(df['wep'] == 6 ,'Frondurchlauf / Continuous freezing rain', inplace=True)

df.to_csv("feature_data_substation_bancroft_labelled.csv")

df[df.wep != 'Blue sky day']

In [ ]:
def create_storm_table(df, id_col, agg_level):
    ''' Summarize storm information from id_col
    '''
    storm_sbs = df.groupby([id_col]+agg_level).agg({
        'valid_datetime':['min','max'],
        })
    
    storm_sbs.columns = ['storm_start_dh','storm_end_dh']
    storm_sbs = storm_sbs.reset_index()
    storm_sbs['duration'] = storm_sbs['storm_end_dh'] - storm_sbs['storm_start_dh']
    storm_sbs['duration_hour'] = storm_sbs['duration'].apply(lambda x: (x.total_seconds()/3600))
    storm_sbs = storm_sbs.rename(columns={id_col:'storm_id'})

    return storm_sbs

### Assign weather / storm ID by cluster label

In [ ]:
# Assign storm id by cluster label
df['valid_datetime'] = pd.to_datetime(df['valid_datetime'])
storm_id_series_lst = []
for cluster_id in df.label2.dropna().unique():
    cluster_id = int(cluster_id)
    dt_series = df.query("label2==%s"%cluster_id).sort_values(['valid_datetime'])['valid_datetime']
    storm_id_series = str(cluster_id)+'_s' + assignStormIdHour(dt_series, 8).map(lambda x: str(x).zfill(3))
    storm_id_series_lst.append(storm_id_series)
# add storm id to df    
df['storm_id'] = pd.concat(storm_id_series_lst)    

### Assign weather / storm ID for all weather / storm days

In [ ]:
# Assign storm id, all storm days
dt_series = df.loc[~df['label2'].isnull()].sort_values(['valid_datetime'])['valid_datetime']
df['storm_id2'] = 's' + assignStormIdHour(dt_series, 8).map(lambda x: str(x).zfill(3))

In [ ]:
df['year'] = df['valid_datetime'].dt.year
df['month'] = df['valid_datetime'].dt.month  

### 7.1. Map storm by cluster to storm_all

In [ ]:
storm = df.groupby(['year','storm_id2','storm_id','label2']).agg({
    'valid_datetime':['min','max'],'min_temp':['min'],'max_windgust':['max'],'max_windspd':['max'],
    'max_snow_density_6':['max'],'max_cumulative_precip':['max'],'max_cumulative_snow':['max'],'max_cumulative_ice':['max'],'wep':['max']
    })

storm.columns = ['storm_start_dh','storm_end_dh','min_temp','max_windgust','max_windspd','max_snow_density','max_cumulative_precip','max_cumulative_snow','max_cumulative_ice','wep']
storm = storm.reset_index()
storm['duration'] = storm['storm_end_dh'] - storm['storm_start_dh']
storm['duration_hour'] = storm['duration'].apply(lambda x: (x.total_seconds()/3600))
storm['label2'] = storm['label2'].map(int)
storm.head()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
storm.query("year==2022 and duration_hour>3").to_csv("Weatherevents for 2022.csv")
storm.query("year==2022 and duration_hour>3")

In [ ]:
storm.query("year==2022 and duration_hour>3").groupby('storm_id2').agg({'label2':list})

In [ ]:
endTime = time.time()
elapsedTime = endTime - startTime
print("Execution time: ", elapsedTime, " seconds")

endTimeCPU = time.process_time()
res =  endTimeCPU - startTimeCPU
print("CPU Execution time: ", res, " seconds")


process = psutil.Process(os.getpid())
print("used non-swapped physical memory (in kiloBytes)", process.memory_info().rss, " kiloBytes")  

print(getrusage(RUSAGE_SELF))
print(getrusage(RUSAGE_SELF).ru_maxrss)